<a href="https://colab.research.google.com/github/serawangari/CGIAR-Root-Volume/blob/main/Liech__CDI_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 30.7 MB/s eta 0:00:00


In [ ]:
# # STEP 1: Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# STEP 2: Import libraries
import os
import numpy as np
import rasterio
import glob
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv2D
import matplotlib.pyplot as plt

# STEP 3: Load TIFFs and metadata
def load_cdi_tiffs(folder_path):
    file_list = sorted(glob.glob(os.path.join(folder_path, "CDI_*.tif")))
    arrays = []
    with rasterio.open(file_list[0]) as src0:
        meta = src0.meta
    for file in file_list:
        with rasterio.open(file) as src:
            arr = src.read(1)
            arrays.append(arr)
    return np.array(arrays), meta
folder = "/content/drive/MyDrive/RodProj"
data, meta = load_cdi_tiffs(folder)

print("Original shape:", data.shape)
print("Any NaNs?:", np.isnan(data).any())

# STEP 4: Replace NaNs and Normalize
data = np.nan_to_num(data, nan=0.0)
data_min = np.min(data)
data_max = np.max(data)
if data_max != data_min:
    data = (data - data_min) / (data_max - data_min)
else:
    data = np.zeros_like(data)

# STEP 5: Create sequences
def create_sequences(data, seq_len=12):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len])
    X = np.array(X)[..., np.newaxis]  # add channel dimension
    y = np.array(y)[..., np.newaxis]
    return X, y

X, y = create_sequences(data, seq_len=12)
print("X shape:", X.shape, "y shape:", y.shape)

# STEP 6: Train/Val/Test split
train_size = int(len(X) * 0.7)
val_size = int(len(X) * 0.15)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

# STEP 7: Clear previous model state
tf.keras.backend.clear_session()

# STEP 8: Define ConvLSTM model with safe settings
model = Sequential([
    ConvLSTM2D(32, (3, 3), activation='relu', return_sequences=True, padding='same', input_shape=X_train.shape[1:]),
    BatchNormalization(),
    ConvLSTM2D(16, (3, 3), activation='relu', return_sequences=False, padding='same'),
    BatchNormalization(),
    Conv2D(1, (1, 1), activation='sigmoid', padding='same')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
model.summary()

# STEP 9: Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=2)

# STEP 10: Evaluate the model
y_pred = model.predict(X_test)

def evaluate(y_true, y_pred):
    y_true_flat = y_true.reshape(-1)
    y_pred_flat = y_pred.reshape(-1)
    print("✅ Evaluation Metrics:")
    print("RMSE:", np.sqrt(mean_squared_error(y_true_flat, y_pred_flat)))
    print("MAE:", mean_absolute_error(y_true_flat, y_pred_flat))
    print("R2 Score:", r2_score(y_true_flat, y_pred_flat))
    print("MAPE:", mean_absolute_percentage_error(y_true_flat, y_pred_flat))

evaluate(y_test, y_pred)

# STEP 11: Predict 2021 and 2022 (24 months ahead)
input_seq = X[-1:]  # last valid sequence
future_preds = []

for i in range(24):
    pred = model.predict(input_seq)
    future_preds.append(pred[0])  # store prediction
    input_seq = np.concatenate([input_seq[:, 1:], pred[:, np.newaxis]], axis=1)

future_preds = np.array(future_preds)  # shape (24, height, width, 1)

# STEP 12: Save predictions month by month
def save_raster(pred_array, meta, filename):
    meta.update(dtype='float32', count=1)
    with rasterio.open(filename, 'w', **meta) as dst:
        dst.write(pred_array.astype(np.float32), 1)

# Make folders for outputs
os.makedirs('/content/Predictions_2021', exist_ok=True)
os.makedirs('/content/Predictions_2022', exist_ok=True)

# Save each month separately
for i in range(24):
    year = 2021 if i < 12 else 2022
    month = (i % 12) + 1
    filename = f"/content/Predictions_{year}/Predicted_CDI_{year}_{month:02d}.tif"
    save_raster(future_preds[i, ..., 0], meta, filename)

print("✅ All monthly predictions saved!")

# Optional: download zipped folders if you want
import shutil
shutil.make_archive('/content/Predictions_2021', 'zip', '/content/Predictions_2021')
shutil.make_archive('/content/Predictions_2022', 'zip', '/content/Predictions_2022')

from google.colab import files
files.download('/content/Predictions_2021.zip')
files.download('/content/Predictions_2022.zip')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Load and prepare data
file_path = '/content/drive/MyDrive/RodProj/MLData.csv'
df = pd.read_csv(file_path)

# Combine Year and Month into datetime
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
df.sort_values('Date', inplace=True)
df.set_index('Date', inplace=True)

# Use only the 'CDI' column
data = df[['CDI']].copy()

# Scale data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Sequence length
n_steps = 12
n_features = 1
n_seq = 1

# Create sequences
X, y = [], []
for i in range(n_steps, len(scaled_data)):
    X.append(scaled_data[i-n_steps:i])
    y.append(scaled_data[i])

X = np.array(X)
y = np.array(y)

# Reshape for ConvLSTM2D
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

# Train/test split
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build ConvLSTM model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 3), activation='relu',
                     input_shape=(n_seq, 1, n_steps, n_features)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16, callbacks=[early_stop])

# Forecasting next 24 months
last_seq = scaled_data[-n_steps:]
forecast_input = last_seq.reshape((1, n_seq, 1, n_steps, n_features))
predictions_scaled = []

for _ in range(24):  # 24 months = 2 years
    pred = model.predict(forecast_input)
    predictions_scaled.append(pred[0, 0])
    last_seq = np.append(last_seq[1:], pred[0, 0])
    forecast_input = last_seq.reshape((1, n_seq, 1, n_steps, n_features))

# Inverse transform
predictions = scaler.inverse_transform(np.array(predictions_scaled).reshape(-1, 1))

# Create date range for forecast
last_date = data.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=24, freq='MS')
forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecast_CDI': predictions.flatten()})
forecast_df.set_index('Date', inplace=True)

# Plot
plt.figure(figsize=(14,6))
plt.plot(data, label='Historical CDI')
plt.plot(forecast_df, label='Forecasted CDI (2021-2022)', linestyle='--')
plt.title('CDI Forecast using ConvLSTM (2021-2022)')
plt.xlabel('Date')
plt.ylabel('CDI')
plt.legend()
plt.grid(True)
plt.show()

# Summary statistics
print("Summary statistics for forecasted CDI (2021 & 2022):")
print(forecast_df.describe())


In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(12, 5))
plt.plot(y_test_inv, label='Actual CDI', marker='o')
plt.plot(y_pred_inv, label='Predicted CDI', marker='x')
plt.title('Actual vs Predicted CDI on Test Set')
plt.xlabel('Time Steps')
plt.ylabel('CDI')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Residuals
residuals = y_test_inv.flatten() - y_pred_inv.flatten()

plt.figure(figsize=(12, 4))
plt.plot(residuals, label='Residuals', color='purple')
plt.hlines(0, xmin=0, xmax=len(residuals), colors='gray', linestyles='dashed')
plt.title('Residual Errors (Actual - Predicted CDI)')
plt.xlabel('Time Steps')
plt.ylabel('Residual')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


##Validation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Load data
cdi_df = pd.read_csv('/content/drive/MyDrive/RodProj/MLData.csv')
met_df = pd.read_csv('/content/drive/MyDrive/RodProj/MetData.csv')

# Merge on Year and Month
merged_df = pd.merge(
    cdi_df, met_df,
    how='inner',
    on=['Year', 'Month']
)

# Create datetime column after merge
merged_df['Date'] = pd.to_datetime(merged_df[['Year', 'Month']].assign(DAY=1))

# Sort and set index
merged_df.sort_values('Date', inplace=True)
merged_df.set_index('Date', inplace=True)

# Normalize CDI, TMPMIN, PRECIP for better visual comparison
scaler = MinMaxScaler()
norm_values = scaler.fit_transform(merged_df[['CDI', 'TMPMIN', 'PRECIP']])
norm_df = pd.DataFrame(norm_values, columns=['CDI', 'TMPMIN', 'PRECIP'], index=merged_df.index)

# --- Plot 1: CDI vs TMPMIN ---
plt.figure(figsize=(14, 5))
plt.plot(norm_df['CDI'], label='CDI', color='blue')
plt.plot(norm_df['TMPMIN'], label='Average Temperature (TMPMIN)', color='orange')
plt.title('CDI vs Min Temperature')
plt.ylabel('Normalized Values')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Plot 2: CDI vs PRECIP ---
plt.figure(figsize=(14, 8))
plt.plot(norm_df['CDI'], label='CDI', color='blue')
plt.plot(norm_df['PRECIP'], label='Precipitation', color='green')
plt.title('CDI vs Precipitation')
plt.ylabel('Normalized Values')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
